In [ ]:
#%pip install openpyxl

In [ ]:
import scanpy as sc
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import median_abs_deviation as mad
import numpy as np
import anndata as adt
from scipy.sparse import csr_matrix
import umap
import scanpy.external as sce

In [ ]:
import warnings 
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [ ]:
path = 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/data.treatment_allconditions_pg_.xlsx'

# Dataset formating

In [ ]:
sheets = pd.read_excel(path, sheet_name=None, header = None)

In [ ]:
cles = list(sheets.keys())

In [ ]:
cles

In [ ]:
adatas = []

for x in cles:
    sheet_part = sheets[x]
    var = sheet_part.iloc[1:, 0:5]  
    obs = pd.DataFrame(sheet_part.iloc[0, 5:])  
    data = sheet_part.iloc[1:, 5:].T

    adata = sc.AnnData(X=data, obs=obs, var=var) #create anndata
    
    #remodele the indexs
    #idx_var = pd.DataFrame(sheet_part.iloc[1:, 3]) 
    adata.obs.index = obs
    
    adata.obs.index = adata.obs.index.map(str)
    adata.obs.index = adata.obs.index.str.replace(r"[()']", "", regex=True)
    adata.obs.index = adata.obs.index.str.replace(";", "")
    adata.obs.index = adata.obs.index.str.replace(",", "")


    idx_var = pd.DataFrame(sheet_part.iloc[1:, 2])  # Index pour var (colonne cible)
    adata.var.index = [str(idx[0]) if isinstance(idx, tuple) else str(idx) for idx in idx_var.iloc[:, 0]]
    adata.var.index = [idx.replace("(", "").replace(")", "").replace("'", "").replace(";", "").replace(",", "") for idx in adata.var.index]
    
    adata.var.columns = sheet_part.iloc[0, 0:5]

    
    adata.obs = adata.obs.drop(columns=[0])
    
    adatas.append(adata)

In [ ]:
adatas[0].var

In [ ]:
np.nan_to_num(adatas[0].X, nan=0.0).min()

In [ ]:
np.nan_to_num(adatas[0].X, nan=0.0).max()

In [ ]:
def load_it(i, adata):
    adata.obs['Id'] = cles[i]
    return adata

In [ ]:
adatas = [load_it(i, ad) for i, ad in enumerate(adatas)]

In [ ]:
from scipy.sparse import csr_matrix
for ad in adatas: 
    
    if isinstance(ad.X, np.ndarray):
         ad.X = ad.X.astype(np.float32)

    if isinstance(ad.X, csr_matrix):
         ad.X = csr_matrix(ad.X, dtype=np.float32)
    # Convertir toutes les colonnes de obs et var en chaînes
    ad.obs = ad.obs.astype(str)
    ad.var = ad.var.astype(str)

    # Convertir les index d'obs et var en chaînes
    ad.obs.index = ad.obs.index.map(str)
    ad.var.index = ad.var.index.map(str)

In [ ]:
for ad in adatas:
    unique_id = ad.obs['Id'][0]
    file_path = os.path.join('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/', f'adata_{unique_id}.h5ad')
    ad.write(file_path)
    print(f"Saved: {file_path}")

# Data exploration

### Understanding of the data and dataset preparation

In [ ]:
adatas = []
data_dir = 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data'

for filename in os.listdir(data_dir):
    if filename.endswith('.h5ad'):
            adata = sc.read_h5ad(os.path.join(data_dir, filename))
            adata.var_names_make_unique()
            adatas.append(adata)

print(f"Nombre de fichiers h5ad chargés avec succès : {len(adatas)}")


In [ ]:
for ad in adatas:
    nbmiss = np.sum(np.isnan(ad.X))/ np.sum(np.size(ad.X)) *100# ad.X contient la matrice de données
    print(f"Pourcentage of NaN values: {nbmiss}%")

In [ ]:
adatas

In [ ]:
adatas[1].X

In [ ]:
all_incorrect_indices = set()

for ad in adatas:
    print(ad.obs['Id'][0])
    ad.var['Genes'] = ad.var['Genes'].astype(str)
    if 'Genes' not in ad.var.columns:
        print("The column doesn't exist in adata.var")
    else:
        # Create a mask for the datas with date name
        date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%m-%d %H:%M:%S', errors='coerce').notna()
        other_date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%d-%m %H:%M:%S', errors='coerce').notna()

    # same for NaN
        nan_mask = ad.var['Genes'].astype(str).str.lower() == 'nan'

    # Combine
        incorrect_entries = date_mask | nan_mask | other_date_mask

        incorrect_genes = ad.var.loc[incorrect_entries, 'Genes']

        print("Incorrect name in column 'Genes' :")
        print(incorrect_genes)
        print(f"\nNombre total d'entrées incorrectes : {incorrect_entries.sum()}")
        print("\nIncorrect distribution :")
        print(incorrect_genes.value_counts())

   
        print("\nIncorrect Index :")
        print(incorrect_genes.index.tolist())
        all_incorrect_indices.update(incorrect_genes.index)


print("\nTous les indices incorrects uniques :")
print(sorted(list(all_incorrect_indices)))
print(f"\nNombre total d'indices incorrects uniques : {len(all_incorrect_indices)}")

In [ ]:
wrong_gene_name = {
"IGA2_HUMAN":"IGA2",
"IGL1_HUMAN":"IGL1",
"SEP10_HUMAN":"SEPTIN10",
"SEP11_HUMAN":"SEPTIN11",
"SEPT2_HUMAN":"SEPTIN2",
"SEPT5_HUMAN":"SEPTIN5",
"SEPT6_HUMAN":"SEPTIN6",
"SEPT7_HUMAN":"SEPTIN7",
"SEPT8_HUMAN":"SEPTIN8",
"SEPT9_HUMAN":"SEPTIN9",
"SYMC_HUMAN":"MARS1",
"SYMM_HUMAN":"MARS2",
"YI025_HUMAN":"YI025",
"YJ005_HUMAN":"YJ005"
}

In [ ]:
def corriger_noms_genes(adatas, corrections):
    for ad in adatas:

        ad.var['Genes'] = ad.var['Genes'].astype(str)
        

        ad.var['Genes'] = ad.var['Genes'].replace(corrections)
        
        date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%m-%d %H:%M:%S', errors='coerce').notna()
        
        mask_a_corriger = date_mask | (ad.var['Genes'].str.lower() == 'nan')
        ad.var.loc[mask_a_corriger, 'Genes'] = ad.var.index[mask_a_corriger].str.split('_').str[0]


corriger_noms_genes(adatas, wrong_gene_name)

In [ ]:
all_incorrect_indices = set()

for ad in adatas:
    print(ad.obs['Id'][0])
    ad.var['Genes'] = ad.var['Genes'].astype(str)
    if 'Genes' not in ad.var.columns:
        print("The column doesn't exist in adata.var")
    else:
        # Create a mask for the datas with date name
        date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%m-%d %H:%M:%S', errors='coerce').notna()
        other_date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%d-%m %H:%M:%S', errors='coerce').notna()

    # same for NaN
        nan_mask = ad.var['Genes'].astype(str).str.lower() == 'nan'

    # Combine
        incorrect_entries = date_mask | nan_mask | other_date_mask

        incorrect_genes = ad.var.loc[incorrect_entries, 'Genes']

        print("Incorrect name in column 'Genes' :")
        print(incorrect_genes)
        print(f"\nNombre total d'entrées incorrectes : {incorrect_entries.sum()}")
        print("\nIncorrect distribution :")
        print(incorrect_genes.value_counts())

   
        print("\nIncorrect Index :")
        print(incorrect_genes.index.tolist())
        all_incorrect_indices.update(incorrect_genes.index)


print("\nTous les indices incorrects uniques :")
print(sorted(list(all_incorrect_indices)))
print(f"\nNombre total d'indices incorrects uniques : {len(all_incorrect_indices)}")

### Differences between datasets

In [ ]:
for ad in adatas:
    ax = sns.histplot(np.nan_to_num(ad.X, nan=0.0).sum(1), bins=100, kde=False, label = ad.obs['Id'][0])
    ax.legend()
#sum of cells for each 

In [ ]:
adatas[1].X

In [ ]:
adata_raw = adt.concat(adatas, join="outer", index_unique="-", merge="unique")

In [ ]:
adata_raw.X = np.nan_to_num(adata_raw.X, nan=0.0)

In [ ]:
sc.pp.highly_variable_genes(adata_raw, n_top_genes=2000, flavor="cell_ranger", batch_key='Id')

In [ ]:
n_batches = adata_raw.var["highly_variable_nbatches"].value_counts()
ax = n_batches.plot(kind="bar")
n_batches
ax.set_xlabel("Nombre de lots")
ax.set_ylabel("Nombre de gènes")
ax.set_title("Distribution des gènes hautement variables à travers les lots")

In [ ]:
adatas[0].var

In [ ]:
all_genes = list(set().union(*[set(adata.var_names) for adata in adatas]))
print(len(all_genes))

presence_matrix = pd.DataFrame(0, index=all_genes, columns=[f"Dataset {ad.obs['Id'][0]}" for ad in adatas])

for adata in adatas:
    presence_matrix.loc[adata.var_names, f"Dataset {adata.obs['Id'][0]}"] = 1  

compte = pd.DataFrame(0, index=all_genes, columns=[f"Seen in {i+1}" for i in range(len(adatas))])
sum_presencematrix = presence_matrix.sum(1)

for gene in all_genes:
    presence_count = sum_presencematrix[gene] 
    compte.loc[gene, f"Seen in {presence_count}"] = 1  


column_sums = compte.sum(axis=0)


plt.figure(figsize=(8, 6))
sns.barplot(x=column_sums.index, y=column_sums.values, palette="viridis")

plt.xlabel("Number of datasets where gene seen")
plt.ylabel("Number of genes")
plt.title("Number of genes shared between datasets")


plt.xticks(rotation=45)

plt.show()

In [ ]:
adatas[1].X

## Erase non detected genes in batch

In [ ]:
protein_diff = set(adatas[2].var_names)
protein_diff2 = set(adatas[3].var_names)
all_protein_diff = (protein_diff-protein_diff2)  ^(protein_diff2-protein_diff)

In [ ]:
adatas_red = [adata.copy() for adata in adatas]
for i,ad in enumerate(adatas_red): 
    adatas_red[i] = ad[:, ~ad.var_names.isin(all_protein_diff)]
adatas_red

In [ ]:
ax = sns.histplot(np.nan_to_num(adatas_red[0].X, nan=0.0).sum(1), bins=100, kde=False, label = adatas_red[0].obs['Id'][0])
ax.legend()

In [ ]:
for ad in adatas_red:
    ax = sns.histplot(np.nan_to_num(ad.X, nan=0.0).sum(1), bins=100, kde=False, label = ad.obs['Id'][0])
    ax.legend()
#sum of cells for each

In [ ]:
adatas_red[1].X

### Normalization tests

In [ ]:
for adata in adatas_red:
    ad = adata.copy()
    ad.X = np.nan_to_num(ad.X, nan=0.0)
    scales_counts = sc.pp.normalize_total(ad, target_sum=None, inplace=False)
    ax = sns.histplot(sc.pp.log1p(scales_counts["X"], copy=True).sum(1), bins=100, kde=False, label = ad.obs['Id'][0])
    ax.legend()

In [ ]:
adatas_red[1].X

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
for adata in adatas_red:
    ad = adata.copy()
    ad.X = np.nan_to_num(ad.X, nan=0.0)
    analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(ad, inplace=False)
    X_fixed = analytic_pearson["X"]
    X_fixed[np.isnan(X_fixed)] = 0 
    ax = sns.histplot(csr_matrix(X_fixed).sum(1), bins=100, kde=False, label = ad.obs['Id'][0])
    ax.legend()

In [ ]:
adatas_red[1].X

In [ ]:
plot_data = []

for adata in adatas_red:
    ad = adata.copy()
    ad.X = np.nan_to_num(ad.X, nan=0.0)  
    analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(ad, inplace=False)
    
    X_fixed = analytic_pearson["X"]
    X_fixed[np.isnan(X_fixed)] = 0  

    plot_data.extend([(x, ad.obs['Id'][0]) for x in csr_matrix(X_fixed).sum(1).A1])

df_plot = pd.DataFrame(plot_data, columns=["Somme Normalisée", "Dataset"])

plt.figure(figsize=(10, 6))
ax = sns.histplot(data=df_plot, x="Somme Normalisée", hue="Dataset", bins=100, kde=False, palette="tab10")

plt.xlabel("Somme des valeurs normalisées")
plt.ylabel("Nombre d'occurrences")
plt.title("Distribution des valeurs normalisées par dataset")

# Afficher le plot
plt.show()


In [ ]:
adatas_red[1].X

### QC metrics

In [ ]:
adatas_red_snan = [adata.copy() for adata in adatas]

In [ ]:
for ad in adatas_red_snan:
    ad.X = np.nan_to_num(ad.X, nan=0.0) 
    

In [ ]:
def qc(adata):
    sc.pp.calculate_qc_metrics(adata, inplace = True, percent_top = [20], log1p=True)
    return adata

In [ ]:
adatas_red_snan = [qc(ad) for ad in adatas_red_snan]

In [ ]:
adatas_red_snan[0].obs

In [ ]:
df = pd.concat([x.obs for x in adatas_red_snan])

In [ ]:
df = df.sort_values('Id')

In [ ]:
value = "pct_counts_in_top_20_genes"

with sns.axes_style("white", rc={"axes.facecolor": (0, 0, 0, 0)}), sns.plotting_context("notebook"):
    g = sns.FacetGrid(df, row="Id", hue="Id", aspect=15, height=0.5, palette="tab20")

    g.map(sns.kdeplot, value, clip_on=False, fill=True, alpha=1, linewidth=1.5)
    g.map(sns.kdeplot, value, clip_on=False, color="w", lw=2)
    g.map(plt.axhline, y=0, lw=2, clip_on=False)

    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, fontweight="bold", color=color,
                ha="left", va="center", transform=ax.transAxes)

    g.map(label, value)

    g.figure.subplots_adjust(hspace=-.6)
    g.set_titles("")
    g.set(yticks=[], ylabel="")
    g.despine(bottom=True, left=True)

    for ax in g.axes.flat:
        ax.axvline(x=df[value].median(), color='r', linestyle='-')

    plt.show()

In [ ]:
value = "n_genes_by_counts"

with sns.axes_style("white", rc={"axes.facecolor": (0, 0, 0, 0)}), sns.plotting_context("notebook"):
    g = sns.FacetGrid(df, row="Id", hue="Id", aspect=15, height=0.5, palette="tab20")

    g.map(sns.kdeplot, value, clip_on=False, fill=True, alpha=1, linewidth=1.5)
    g.map(sns.kdeplot, value, clip_on=False, color="w", lw=2)
    g.map(plt.axhline, y=0, lw=2, clip_on=False)

    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, fontweight="bold", color=color,
                ha="left", va="center", transform=ax.transAxes)

    g.map(label, value)

    g.figure.subplots_adjust(hspace=-.6)
    g.set_titles("")
    g.set(yticks=[], ylabel="")
    g.despine(bottom=True, left=True)

    for ax in g.axes.flat:
        ax.axvline(x=df[value].median(), color='r', linestyle='-')

    plt.show()

In [ ]:
value = "total_counts"

with sns.axes_style("white", rc={"axes.facecolor": (0, 0, 0, 0)}), sns.plotting_context("notebook"):
    g = sns.FacetGrid(df, row="Id", hue="Id", aspect=15, height=0.5, palette="tab20")

    g.map(sns.kdeplot, value, clip_on=False, fill=True, alpha=1, linewidth=1.5)
    g.map(sns.kdeplot, value, clip_on=False, color="w", lw=2)
    g.map(plt.axhline, y=0, lw=2, clip_on=False)

    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, fontweight="bold", color=color,
                ha="left", va="center", transform=ax.transAxes)

    g.map(label, value)

    g.figure.subplots_adjust(hspace=-.6)
    g.set_titles("")
    g.set(yticks=[], ylabel="")
    g.despine(bottom=True, left=True)

    for ax in g.axes.flat:
        ax.axvline(x=df[value].median(), color='r', linestyle='-')

    plt.show()

In [ ]:
adatas_red[1].X

### Highly var proteins btw batches post dimension reduction

In [ ]:
adata_red = adt.concat(adatas_red, join="outer", index_unique="-", merge="unique")

In [ ]:
adata_red.X = np.nan_to_num(adata_red.X, nan=0.0)

In [ ]:
adatas_red[1].X

In [ ]:
sc.pp.highly_variable_genes(adata_red, n_top_genes=2000, flavor="cell_ranger", batch_key='Id')

In [ ]:
n_batches = adata_red.var["highly_variable_nbatches"].value_counts()
ax = n_batches.plot(kind="bar")
n_batches
ax.set_xlabel("Nombre de lots")
ax.set_ylabel("Nombre de gènes")
ax.set_title("Distribution des gènes hautement variables à travers les lots")

### Shared proteins btw batches

In [ ]:
all_genes = list(set().union(*[set(adata.var_names) for adata in adatas_red]))
print(len(all_genes))

presence_matrix = pd.DataFrame(0, index=all_genes, columns=[f"Dataset {ad.obs['Id'][0]}" for ad in adatas_red])

for adata in adatas_red:
    presence_matrix.loc[adata.var_names, f"Dataset {adata.obs['Id'][0]}"] = 1  

compte = pd.DataFrame(0, index=all_genes, columns=[f"Seen in {i+1}" for i in range(len(adatas_red))])
sum_presencematrix = presence_matrix.sum(1)

for gene in all_genes:
    presence_count = sum_presencematrix[gene] 
    compte.loc[gene, f"Seen in {presence_count}"] = 1  


column_sums = compte.sum(axis=0)


plt.figure(figsize=(8, 6))
sns.barplot(x=column_sums.index, y=column_sums.values, palette="viridis")

plt.xlabel("Number of datasets where gene seen")
plt.ylabel("Number of genes")
plt.title("Number of genes shared between datasets")


plt.xticks(rotation=45)

plt.show()

In [ ]:
os.makedirs('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/Formated_data',exist_ok = True)

In [ ]:
for ad in adatas_red:
    unique_id = ad.obs['Id'][0]
    file_path = os.path.join('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/Formated_data', f'adata_{unique_id}.h5ad')
    ad.write(file_path)
    print(f"Saved: {file_path}")

In [ ]:
adatas_red[1].X

In [ ]:
adatas_red = []
data_dir = 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/Formated_data'

for filename in os.listdir(data_dir):
    if filename.endswith('.h5ad'):
            adata = sc.read_h5ad(os.path.join(data_dir, filename))
            adata.var_names_make_unique()
            adatas_red.append(adata)

print(f"Nombre de fichiers h5ad chargés avec succès : {len(adatas_red)}")

In [ ]:
adatas_red

In [ ]:
adatas_red[1].X

In [ ]:
print(type(adatas_red[0].X))

In [ ]:
np.isnan(adatas_red[1].X).any()

In [ ]:
np.nan_to_num(adatas_red[0].X, nan=0.0).min()

In [ ]:
np.nan_to_num(adatas_red[0].X, nan=0.0).max()

## Imputation of data with MissForest

In [ ]:
from missforest import MissForest

In [ ]:
nan_percentage = np.isnan(adatas_red[0].X).mean()
print(f"Proportion de NaN dans X : {nan_percentage:.2%}")

In [ ]:
#os.makedirs( 'G:/Data processing pipeline 0.1 Yohan/scP/Output/missforest_completed',exist_ok = True)

In [ ]:
#for ad in adatas_red:
    
   # miss_forest = MissForest()
   # X = ad.X.copy()
   # X = X.astype(np.float64)
    # Impute away
   # X_imputed = miss_forest.fit_transform(X)
   # ad.X = X_imputed
   # print(ad)
   # unique_id = ad.obs['Id'][0]
   # file_path = os.path.join('G:/Data processing pipeline 0.1 Yohan/scP/Output/missforest_completed', f'adata_{unique_id}.h5ad')
   # ad.write(file_path)
   # print(f"Saved: {file_path}")


In [ ]:
for ad in adatas_red:
    nan_percentage = np.isnan(ad.X).mean()
    print(f"Proportion de NaN dans X : {nan_percentage:.2%}")

In [ ]:
import time

In [ ]:
os.makedirs( 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/missforest_completed',exist_ok = True)

In [ ]:
from missforest import MissForest


for ad in adatas_red:
    mf = MissForest()
    X = ad.X.copy()
    X = X.astype(np.float64)

    X_df = pd.DataFrame(X)
    X_imputed = mf.fit_transform(X_df)
    X_imputed = X_imputed.to_numpy()
    try:
       print("Erreur OOB :", mf.oob_imputation_error_) 
    except:
        print('Error OOB not found')

    ad.X = X_imputed.astype(np.float32)

    print(ad)
    unique_id = ad.obs['Id'][0]
    time.sleep(1)

    file_path = os.path.join('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/missforest_completed', f'adata_{unique_id}.h5ad')
    
    try:
        ad.write(file_path)
        print(f"Saved: {file_path}")
        
    except Exception as e:
        print(f"Erreur lors de la sauvegarde de {file_path} : {e}")
        

In [ ]:
from joblib.externals.loky.backend.context import cpu_count
print(cpu_count(only_physical_cores=True))

In [ ]:
adatas_red[0].X

## Evaluation of the effect of missforest

In [ ]:
adatas_red = []
data_dir = 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/missforest_completed'

for filename in os.listdir(data_dir):
    if filename.endswith('.h5ad'):
            adata = sc.read_h5ad(os.path.join(data_dir, filename))
            adata.var_names_make_unique()
            adatas_red.append(adata)

print(f"Nombre de fichiers h5ad chargés avec succès : {len(adatas_red)}")

In [ ]:
adatas_red

In [ ]:
np.nan_to_num(adatas_red[0].X, nan=0.0).min()

In [ ]:
X_no_nan = np.nan_to_num(adatas_red[5].X, nan=0.0)

# Compter les valeurs inférieures à 0
count_negatives = (X_no_nan < 0).sum()

print(f"Nombre de valeurs inférieures à 0 : {count_negatives}")

In [ ]:
for nb in range(len(adatas_red)):
    
    X_no_nan = np.nan_to_num(adatas_red[nb].X, nan=0.0)

# Trouver les indices des valeurs négatives
    negative_mask = X_no_nan < 0

# Obtenir les indices des gènes (ou variables) avec des valeurs négatives
    negative_genes_indices = np.where(negative_mask.any(axis=0))[0]

# Récupérer les noms des gènes/variables associés à ces indices
    negative_genes = adatas_red[nb].var_names[negative_genes_indices]

# Afficher les gènes avec des valeurs négatives
    print(f"Gènes avec des valeurs négatives : {negative_genes}")

In [ ]:
for ad in adatas_red:
    ax = sns.histplot(np.nan_to_num(ad.X, nan=0.0).sum(1), bins=100, kde=False, label = ad.obs['Id'][0])
    ax.legend()

In [ ]:
adata = adt.concat(adatas_red, join="outer", index_unique="-", merge="unique")
adata.layers['raw'] = adata.X
adata.X = np.nan_to_num(adata.X, nan=0.0)
adata.X = np.where(adata.X < 0, 0, adata.X)
adata.layers['raw_woutnan'] = adata.X

In [ ]:
adata.X.min()

In [ ]:
adata

In [ ]:
os.makedirs('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/concat',exist_ok = True)
file_path = os.path.join('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/concat', 'adata_concat_normalized.h5ad')
adata.write_h5ad(file_path)
print(f"Saved: {file_path}")

In [ ]:
adata = sc.read_h5ad('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/concat/adata_concat_normalized.h5ad')

In [ ]:
limit_gene_acceptance = 0 # number considered as not commun gene

In [ ]:
all_genes = list(set().union(*[set(ad.var_names) for ad in adatas_red]))
print(len(all_genes))



presence_matrix = pd.DataFrame(0, index=all_genes, columns=[f"Dataset {ad.obs['Id'][0]}" for ad in adatas_red])

for ad in adatas_red:
    presence_matrix.loc[ad.var_names, f"Dataset {ad.obs['Id'][0]}"] = 1  

compte = pd.DataFrame(0, index=all_genes, columns=[f"Seen in {i+1}" for i in range(len(adatas_red))])
sum_presencematrix = presence_matrix.sum(1)

commun_gene = []
for gene in all_genes:
    presence_count = sum_presencematrix[gene] 
    if presence_count>limit_gene_acceptance:
        commun_gene.append(gene)
    compte.loc[gene, f"Seen in {presence_count}"] = 1  

adata.var['Common']=adata.var_names.isin(commun_gene)

column_sums = compte.sum(axis=0)


plt.figure(figsize=(8, 6))
sns.barplot(x=column_sums.index, y=column_sums.values, palette="viridis")

plt.xlabel("Number of datasets where gene seen")
plt.ylabel("Number of genes")
plt.title("Number of genes shared between datasets")


plt.xticks(rotation=45)

plt.show()

In [ ]:
adata

### Test umap with log1p norm and commun genes

In [ ]:
adata_common = adata[:, adata.var['Common']]
adata_common.layers["raw_counts"] = adata_common.X.copy()

In [ ]:
adata_common.X = np.nan_to_num(adata_common.X, nan=0.0)
scales_counts = sc.pp.normalize_total(adata_common, target_sum=None, inplace=False)
# log1p transform
adata_common.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)
adata_common.X = adata_common.layers['log1p_norm']
sc.pp.pca(adata_common)
sc.pp.neighbors(adata_common, use_rep = 'X_pca')
sc.tl.leiden(adata_common, resolution = 2, key_added = 'overcluster')
sc.tl.umap(adata_common)
sc.pl.umap(
    adata_common,
    color=['Id']
)

In [ ]:
# lop1p post MissForest

In [ ]:
for ada in adatas_red:
    ad = ada.copy()
    ad.X = np.nan_to_num(ad.X, nan=0.0)
    scales_counts = sc.pp.normalize_total(ad, target_sum=None, inplace=False)
    ax = sns.histplot(sc.pp.log1p(scales_counts["X"], copy=True).sum(1), bins=100, kde=False, label = ad.obs['Id'][0])
    ax.legend()

In [ ]:
#did not work with log1p and need to use one more time nan to num

In [ ]:
adata_common.X = np.nan_to_num(adata_common.X, nan=0.0)
scales_counts = sc.pp.normalize_total(adata_common, target_sum=None, inplace=False)
# log1p transform
adata_common.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)
adata_common.X = adata_common.layers['log1p_norm']
adata_common.X = np.nan_to_num(adata_common.X, nan=0.0)
sc.pp.pca(adata_common)
sc.pp.neighbors(adata_common, use_rep = 'X_pca')
sc.tl.leiden(adata_common, resolution = 2, key_added = 'overcluster')
sc.tl.umap(adata_common)
sc.pl.umap(
    adata_common,
    color=['Id']
)

### Effect of the normalization using pearson residue

In [ ]:
plot_data = []

for ade in adatas_red:
    ad = ade.copy()
    #ad.X = np.nan_to_num(ad.X, nan=0.0)  
    analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(ad, inplace=False)
    
    X_fixed = analytic_pearson["X"]
    
    plot_data.extend([(x, ad.obs['Id'][0]) for x in csr_matrix(X_fixed).sum(1).A1])

df_plot = pd.DataFrame(plot_data, columns=["Normalized Sum", "Dataset"])

plt.figure(figsize=(10, 6))
ax = sns.histplot(data=df_plot, x="Normalized Sum", hue="Dataset", bins=100, kde=False, palette="tab10")

plt.xlabel("Sum of the normalized values")
plt.ylabel("Number of cells")
plt.title("Distribution of the Pearson Residue Normalization")

# Afficher le plot
plt.show()

In [ ]:
adata_common.X =adata_common.layers["raw_counts"]
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(adata_common, inplace=False)
X_fixed = analytic_pearson["X"]
X_fixed[np.isnan(X_fixed)] = 0 
adata_common.layers["analytic_pearson_residuals_train"] = csr_matrix(X_fixed)
adata_common.X = adata_common.layers["analytic_pearson_residuals_train"]
sc.pp.pca(adata_common)
sc.pp.neighbors(adata_common, use_rep = 'X_pca')
sc.tl.leiden(adata_common, resolution = 3, key_added = 'overcluster')
sc.tl.umap(adata_common)
sc.pl.umap(
    adata_common,
    color=['overcluster','Id']
)

In [ ]:
adata

In [ ]:
import scanpy.external as sce
from scipy.sparse import csr_matrix

In [ ]:
adata.X =adata.layers["raw_woutnan"]
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(adata, inplace=False)
X_fixed = analytic_pearson["X"]
X_fixed[np.isnan(X_fixed)] = 0 
adata.layers["analytic_pearson_residuals_train"] = csr_matrix(X_fixed)
adata.X = adata.layers["analytic_pearson_residuals_train"]
sc.pp.pca(adata)
sce.pp.scanorama_integrate(adata, 'Id', verbose =1)

In [ ]:
sc.pl.pca_variance_ratio(adata_common, log=True)

In [ ]:
sc.pp.neighbors(adata, use_rep = 'X_scanorama')
sc.tl.leiden(adata, resolution = 3, key_added = 'overcluster')
sc.tl.umap(adata)
sc.pl.umap(
    adata,
    color=['overcluster','Id']
)

In [ ]:
np.random.seed(1)
ri = np.random.permutation(list(range(adata.shape[0])))

sc.pl.umap(adata[ri,:], color = ['Id'], vmin = .5, size = 20)

In [ ]:
#ciliated
sc.pl.umap(adata, color=['TPPP3_HUMAN','TBB4B_HUMAN','SNTAN_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
#basal
sc.pl.umap(adata, color=['K1C15_HUMAN','K2C5_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
#secretory club
sc.pl.umap(adata, color=['UTER_HUMAN','BPIB1_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
#ciliated
sc.pl.umap(adata, color=['TPPP3_HUMAN','TBB4B_HUMAN','SNTAN_HUMAN','CCD17_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
import umap
umap_3d = umap.UMAP(n_components=3, random_state=42)

adata.obsm['X_umap_3d'] = umap_3d.fit_transform(adata.obsm['X_scanorama'])

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

cell_types = list(adata.obs['Id'].unique())
color_palette = px.colors.qualitative.Set1[:len(cell_types)]
color_dict = {ct: color for ct, color in zip(cell_types, color_palette)}


fig = go.Figure(data=[go.Scatter3d(
    x=adata.obsm['X_umap_3d'][:, 0],
    y=adata.obsm['X_umap_3d'][:, 1],
    z=adata.obsm['X_umap_3d'][:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color=[color_dict.get(cell_type, '#000000') for cell_type in adata.obs['Id']],
        opacity=0.8
    ),
    text=adata.obs['Id'],
    hoverinfo='text'
)])
# Mettez à jour la mise en page
fig.update_layout(
    scene=dict(aspectmode="data"),
    title="3D UMAP "
)

fig.show()

In [ ]:
sc.pl.pca_scatter(adata, color="Id")

In [ ]:
explained_variance = adata.uns["pca"]["variance_ratio"] * 100  # Conversion en pourcentage
for i, var in enumerate(explained_variance[:10]):  # Afficher les 10 premières composantes
    print(f"PC{i+1}: {var:.2f}% de variance expliquée")

In [ ]:
sc.pl.pca_scatter(adata, components=["3,4"], color="Id")

In [ ]:
adata

In [ ]:
os.makedirs('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/normalized_pearson_data',exist_ok = True)
file_path = os.path.join('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/normalized_pearson_data', 'adata_concat_normalized.h5ad')
adata.write_h5ad(file_path)
print(f"Saved: {file_path}")

# Test avec données pré miss forest et scanorama

In [ ]:
adatas_red = []
data_dir = 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/Formated_data'

for filename in os.listdir(data_dir):
    if filename.endswith('.h5ad'):
            adata = sc.read_h5ad(os.path.join(data_dir, filename))
            adata.var_names_make_unique()
            adatas_red.append(adata)

print(f"Nombre de fichiers h5ad chargés avec succès : {len(adatas_red)}")

In [ ]:
plot_data = []

for ade in adatas_red:
    ad = ade.copy()
    ad.X = np.nan_to_num(ad.X, nan=0.0)  
    analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(ad, inplace=False)
    
    X_fixed = analytic_pearson["X"]
    
    plot_data.extend([(x, ad.obs['Id'][0]) for x in csr_matrix(X_fixed).sum(1).A1])

df_plot = pd.DataFrame(plot_data, columns=["Normalized Sum", "Dataset"])

plt.figure(figsize=(10, 6))
ax = sns.histplot(data=df_plot, x="Normalized Sum", hue="Dataset", bins=100, kde=False, palette="tab10")

plt.xlabel("Sum of the normalized values")
plt.ylabel("Number of cells")
plt.title("Distribution of the Pearson Residue Normalization")

# Afficher le plot
plt.show()

In [ ]:
adata1 = adt.concat(adatas_red, join="outer", index_unique="-", merge="unique")
adata1.layers['raw'] = adata1.X
adata1.X = np.nan_to_num(adata1.X, nan=0.0)

In [ ]:
adata1.obs['expression_sum'] = np.array(adata1.X.sum(axis=1)).flatten()

In [ ]:
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(adata1, inplace=False)
X_fixed = analytic_pearson["X"]
X_fixed[np.isnan(X_fixed)] = 0 
adata1.layers["analytic_pearson_residuals_train"] = csr_matrix(X_fixed)
adata1.X = adata1.layers["analytic_pearson_residuals_train"]
sc.pp.pca(adata1)
sce.pp.scanorama_integrate(adata1, 'Id', verbose =1)

In [ ]:
sc.pl.pca_variance_ratio(adata1, log=True)

In [ ]:
sc.pp.neighbors(adata1, use_rep = 'X_scanorama')
sc.tl.leiden(adata1, resolution = 2, key_added = 'overcluster')
sc.tl.umap(adata1)
sc.pl.umap(
    adata1,
    color=['overcluster']
)

In [ ]:
np.random.seed(1)
ri = np.random.permutation(list(range(adata1.shape[0])))

sc.pl.umap(adata1[ri,:], color = ['expression_sum','Id','overcluster'], vmin = .5, size = 20)

In [ ]:
adata1

In [ ]:
sc.pl.umap(adata1, color=['TPPP3_HUMAN'],legend_loc="on data",
)

In [ ]:
#ciliated
sc.pl.umap(adata1, color=['TPPP3_HUMAN','TBB4B_HUMAN','SNTAN_HUMAN','CCD17_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
#basal
sc.pl.umap(adata1, color=['K1C15_HUMAN','K2C5_HUMAN','MUC4_HUMAN','MUC5B_HUMAN','MUC5A_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
#secretory club
sc.pl.umap(adata1, color=['UTER_HUMAN','BPIB1_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
#secretory club
sc.pl.umap(adata1, color=['TRFL_HUMAN','CADH1_HUMAN','MUC5A_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
sc.tl.rank_genes_groups(adata1, groupby = 'overcluster')
marks = sc.get.rank_genes_groups_df(adata1, group = None)

In [ ]:
marks[(marks.group == '14') ].sort_values('logfoldchanges', ascending = False).head(60)

In [ ]:
ax = sc.pl.umap(adata1, palette = 'lightgrey', show = False)
sc.pl.umap(adata1[adata1.obs.overcluster == '10'], color = 'overcluster',size =45, ax = ax, legend_loc = None, palette = 'k')

In [ ]:
marks

### Dendro with data non imputed

In [ ]:
#marks['pvals'].mean()

In [ ]:
#dendr_df = marks.pivot(index="group", columns="names", values="logfoldchanges")

In [ ]:
#dendr_df

In [ ]:
#nan_percentage = dendr_df.isnan().mean()
#print(f"Proportion de NaN dans X : {nan_percentage:.2%}")

In [ ]:
total_nan_percentage = dendr_df.isna().mean().mean()
print(f"Proportion totale de NaN dans le DataFrame : {total_nan_percentage:.2%}")

In [ ]:
#dendr_df_clean = dendr_df.loc[:, (dendr_df > 2).any(axis=0)]

In [ ]:
#dendr_df_clean

In [ ]:
total_nan_percentage = dendr_df_clean.isna().mean().mean()
print(f"Proportion totale de NaN dans le DataFrame : {total_nan_percentage:.2%}")

# only the pos expressed genes

In [ ]:
import scipy.cluster.hierarchy as sch
import scipy.spatial.distance as ssd

In [ ]:
def ranking_evaluation_cluster(marks_cluster, size_rk):
    selected_genes = marks_cluster['names'][:size_rk]
    if marks_cluster.loc[marks_cluster['names'].isin(selected_genes), 'logfoldchanges'].isna().any():
        raise ValueError("Error; NaN values found in the selected proteins")

    ranking_id = marks_cluster['group'].unique()[0]
    rankings_cluster = {f"{ranking_id}": {prot: i + 1 for i, prot in enumerate(selected_genes)}}
    return rankings_cluster

In [ ]:
def ranking_evaluation(marks, size_rk):
    # do not allow NaN values
    ranking = {}
    for num in marks['group'].unique():
        marks_cluster = marks[(marks.group == f"{num}") ].sort_values('logfoldchanges', ascending = False)
        rankings_cluster = ranking_evaluation_cluster(marks_cluster, size_rk)
        ranking.update(rankings_cluster)
    return ranking

In [ ]:
def common(ranking):
    all_genes = list(set.union(*[set(list(ranking[cluster].keys())) for cluster in list(ranking.keys())]))

    presence_matrix = pd.DataFrame(0, index=all_genes, columns=[f"Dataset {cluster}" for cluster in list(ranking.keys())])

    for cluster in list(ranking.keys()):
        presence_matrix.loc[list(ranking[cluster].keys()), f"Dataset {cluster}"] = 1

    compte = pd.DataFrame(0, index=all_genes, columns=[f"Seen in {i+1} clusters" for i in range(len(list(ranking.keys())))])
    sum_presencematrix = presence_matrix.sum(1)
    for gene in all_genes:
        presence_count = sum_presencematrix[gene] 
        compte.loc[gene, f"Seen in {presence_count} clusters"] = 1  


    column_sums = compte.sum(axis=0)


    plt.figure(figsize=(8, 6))
    sns.barplot(x=column_sums.index, y=column_sums.values, palette="viridis")

    plt.xlabel("Number of datasets where proteins seen")
    plt.ylabel("Number of proteins")
    plt.title("Number of proteins shared between datasets")


    plt.xticks(rotation=45)

    plt.show()

In [ ]:
def ranking_dendrogramme(marks, size_rk):
    ranking = ranking_evaluation(marks, size_rk)
    common(ranking)

    all_genes = list(set.union(*[set(list(ranking[cluster].keys())) for cluster in list(ranking.keys())]))
    #matrix_rk = pd.DataFrame(np.nan, index=[f"Dataset {cluster}" for cluster in list(ranking.keys())], columns=all_genes)
   # print(matrix_rk)
    data = {}
    
    for cluster_id, proteins in ranking.items():
        row = {gene: np.nan for gene in all_genes}
        for protein, rank in proteins.items():
            row[protein] = rank
        data[cluster_id] = row
    
    df_ranking = pd.DataFrame(data).T
    df_ranking.fillna(df_ranking.max().max()+1,inplace=True)
    dist_matrix = ssd.squareform(ssd.pdist(df_ranking, metric="correlation"))
    linkage_matrix = sch.linkage(dist_matrix, method="ward")
    
    plt.figure(figsize=(10, 5))
    sch.dendrogram(linkage_matrix)
    plt.title("Dendrogramm of the clusters")
    plt.xlabel("Clusters")
    plt.ylabel("Distance")
    plt.show()

In [ ]:
def custom_dendrogram(*args, **kwargs):
    dendro_data = sch.dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        for icoord, dcoord in zip(dendro_data['icoord'], dendro_data['dcoord']):
            x_coord = 0.5 * sum(icoord[1:3])  
            height = dcoord[1]  
            plt.plot(x_coord, height, 'ro') 
            plt.annotate(f"{height:.3g}", (x_coord, height), xytext=(0, -8),
                         textcoords='offset points', va='top', ha='center')  

    return dendro_data

In [ ]:
def evolution_number_cluster(linkage_matrix):
    distances = linkage_matrix[:, 2]  
    max_distance = max(distances)
    cluster_counts = []

    for distance in distances:
        clusters = sch.fcluster(linkage_matrix, t=distance, criterion='distance')
        cluster_counts.append(len(np.unique(clusters)))

# print of the figure
    plt.figure(figsize=(8, 6))
    plt.plot(distances, cluster_counts, marker='o')
    plt.xlabel('Distance')
    plt.ylabel('Number of clusters')
    plt.title('Evolution of the number of cluster')
    plt.grid(True)
    plt.show()


In [ ]:
def evolution_metacluster(linkage_matrix,list_cluster):

    distances = np.concatenate(([0], linkage_matrix[:, 2]))
    cluster_counts = []
    single_elements_counts = []

    n_elements = len(list_cluster)
    for distance in distances:
        clusters = np.concatenate(([0],sch.fcluster(linkage_matrix, t=distance, criterion='distance')))
        cluster_sizes = np.bincount(clusters)
    
        valid_clusters = np.sum(cluster_sizes >= 2)
        cluster_counts.append(valid_clusters)
    
        single_elements = np.sum(cluster_sizes == 1)
        single_elements_counts.append(single_elements)

    plt.figure(figsize=(8, 6))
    plt.plot(distances,cluster_counts, marker='o', label='MetaCluster')
    plt.xlabel('Distance')
    plt.ylabel('Number of MetaCluster')
    plt.title('Evolution of the number of MetaCluster')
    plt.grid(True)
    plt.legend()
    plt.show()

    plt.figure(figsize=(8, 6))
    plt.plot(distances, single_elements_counts, marker='x', color='green', label='Number of clusters out of MetaCluster')
    plt.xlabel('Distance')
    plt.ylabel('Number of clusters out of MetaCluster')
    plt.title('Evolution of the Number of clusters out of MetaCluster depending of the distance')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
def ranking_dendrogramme(marks, size_rk):
    ranking = ranking_evaluation(marks, size_rk)
    common(ranking)

    all_genes = list(set.union(*[set(list(ranking[cluster].keys())) for cluster in list(ranking.keys())]))
    data = {}
    list_cluster = []
    for cluster_id, proteins in ranking.items():
        list_cluster.append(cluster_id)
        row = {gene: np.nan for gene in all_genes}
        for protein, rank in proteins.items():
            row[protein] = rank
        data[cluster_id] = row

    df_ranking = pd.DataFrame(data).T
    df_ranking.fillna(df_ranking.max().max()+1,inplace=True)
    dist_matrix = ssd.squareform(ssd.pdist(df_ranking, metric="correlation"))
    linkage_matrix = sch.linkage(dist_matrix, method="ward")

    evolution_number_cluster(linkage_matrix)
    evolution_metacluster(linkage_matrix,list_cluster)
    
    plt.figure(figsize=(10, 5))
    plt.figure(figsize=(10, 4))
    dendro_data = custom_dendrogram(linkage_matrix)
    plt.title("Dendrogramm of the clusters")
    plt.xlabel("Clusters")
    plt.ylabel("Distance")
    plt.show()

In [ ]:
ranking_dendrogramme(marks, 200)

In [ ]:
def common_prot_clusters(marks, size_rk,cluster_group,ranking):
    common_prot = list(set.intersection(*[set(list(ranking[cluster].keys())) for cluster in cluster_group]))
    return common_prot

In [ ]:
def tab_sim_prot(marks, size_rk,cluster_group):
    ranking = ranking_evaluation(marks, size_rk)
    common_prot = common_prot_clusters(marks, size_rk,cluster_group,ranking)
    print(common_prot)
    data = {}
    
    for cluster_id, proteins in ranking.items():
        if cluster_id in cluster_group:
            row = {prot: np.nan for prot in common_prot}
            for protein, rank in proteins.items():
                if protein in common_prot:
                    row[protein] = rank
            data[cluster_id] = row
    
    df_ranking = pd.DataFrame(data).T
    df_ranking.fillna(df_ranking.max().max()+1,inplace=True)
    return df_ranking

In [ ]:
cluster_group = ['3','8']

In [ ]:
cluster_group =['10', '2', '5','13','14','16']

In [ ]:
tab_sim_prot(marks, 200,cluster_group)

# Neg prot expression (not interesting)

In [ ]:
def ranking_evaluation_cluster_neg(marks_cluster, size_rk):
    selected_genes = marks_cluster['names'][:size_rk]
    if marks_cluster.loc[marks_cluster['names'].isin(selected_genes), 'logfoldchanges'].isna().any():
        raise ValueError("Error; NaN values found in the selected proteins")

    ranking_id = marks_cluster['group'].unique()[0]
    rankings_cluster = {f"{ranking_id}": {prot: i + 1 for i, prot in enumerate(selected_genes)}}
    return rankings_cluster

In [ ]:
def ranking_evaluation_neg(marks, size_rk):
    # do not allow NaN values
    ranking = {}
    for num in marks['group'].unique():
        marks_cluster = marks[(marks.group == f"{num}") ].sort_values('logfoldchanges', ascending = True)
        rankings_cluster = ranking_evaluation_cluster_neg(marks_cluster, size_rk)
        ranking.update(rankings_cluster)
    return ranking

In [ ]:
def common(ranking):
    all_genes = list(set().union(*[set(list(ranking[cluster].keys())) for cluster in list(ranking.keys())]))

    presence_matrix = pd.DataFrame(0, index=all_genes, columns=[f"Dataset {cluster}" for cluster in list(ranking.keys())])

    for cluster in list(ranking.keys()):
        presence_matrix.loc[list(ranking[cluster].keys()), f"Dataset {cluster}"] = 1

    compte = pd.DataFrame(0, index=all_genes, columns=[f"Seen in {i+1} clusters" for i in range(len(list(ranking.keys())))])
    sum_presencematrix = presence_matrix.sum(1)
    for gene in all_genes:
        presence_count = sum_presencematrix[gene] 
        compte.loc[gene, f"Seen in {presence_count} clusters"] = 1  


    column_sums = compte.sum(axis=0)


    plt.figure(figsize=(8, 6))
    sns.barplot(x=column_sums.index, y=column_sums.values, palette="viridis")

    plt.xlabel("Number of datasets where proteins seen")
    plt.ylabel("Number of proteins")
    plt.title("Number of proteins shared between datasets")


    plt.xticks(rotation=45)

    plt.show()

In [ ]:
def custom_dendrogram(*args, **kwargs):
    dendro_data = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        for icoord, dcoord in zip(dendro_data['icoord'], dendro_data['dcoord']):
            x_coord = 0.5 * sum(icoord[1:3])  
            height = dcoord[1]  
            plt.plot(x_coord, height, 'ro') 
            plt.annotate(f"{height:.3g}", (x_coord, height), xytext=(0, -8),
                         textcoords='offset points', va='top', ha='center')  

    return dendro_data

In [ ]:
def ranking_dendrogramme_neg(marks, size_rk):
    ranking = ranking_evaluation_neg(marks, size_rk)
    common(ranking)

    all_genes = list(set().union(*[set(list(ranking[cluster].keys())) for cluster in list(ranking.keys())]))
    #matrix_rk = pd.DataFrame(np.nan, index=[f"Dataset {cluster}" for cluster in list(ranking.keys())], columns=all_genes)
   # print(matrix_rk)
    data = {}
    
    for cluster_id, proteins in ranking.items():
        row = {gene: np.nan for gene in all_genes}
        for protein, rank in proteins.items():
            row[protein] = rank
        data[cluster_id] = row
    
    df_ranking = pd.DataFrame(data).T
    df_ranking.fillna(df_ranking.max().max()+1,inplace=True)
    dist_matrix = ssd.squareform(ssd.pdist(df_ranking, metric="correlation"))
    linkage_matrix = sch.linkage(dist_matrix, method="ward")
    
    plt.figure(figsize=(10, 5))
    plt.figure(figsize=(10, 4))
    dendro_data = custom_dendrogram(linkage_matrix)
    plt.title("Dendrogramm of the clusters")
    plt.xlabel("Clusters")
    plt.ylabel("Distance")
    plt.show()

In [ ]:
ranking_dendrogramme_neg(marks, 200)

In [ ]:
ax = sc.pl.umap(adata1, palette = 'lightgrey', show = False)
adata_clusters = adata1[adata1.obs['overcluster'].isin(['10', '2', '5','13','14','16'])]

sc.pl.umap(adata_clusters, 
           color='overcluster',  
           size=45,              
           ax=ax,               
           legend_loc=None     
          )  

plt.show()

In [ ]:
ax = sc.pl.umap(adata1, palette = 'lightgrey', show = False)
adata_clusters = adata1[adata1.obs['overcluster'].isin(['1', '9', '17','0','18','22','15','20'])]

sc.pl.umap(adata_clusters, 
           color='overcluster',  
           size=45,              
           ax=ax,               
           legend_loc=None     
          )  

plt.show()

In [ ]:
ax = sc.pl.umap(adata1, palette = 'lightgrey', show = False)
adata_clusters = adata1[adata1.obs['overcluster'].isin(['4', '11', '6','7','12','19','21'])]

sc.pl.umap(adata_clusters, 
           color='overcluster',  
           size=45,              
           ax=ax,               
           legend_loc=None     
          )  

plt.show()

In [ ]:
ax = sc.pl.umap(adata1, palette = 'lightgrey', show = False)
adata_clusters = adata1[adata1.obs['overcluster'].isin(['3','8'])]

sc.pl.umap(adata_clusters, 
           color='overcluster',  
           size=45,              
           ax=ax,               
           legend_loc=None     
          )  

plt.show()

In [ ]:
umap_3d = umap.UMAP(n_components=3, random_state=42)

adata1.obsm['X_umap_3d'] = umap_3d.fit_transform(adata1.obsm['X_scanorama'])

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

cell_types = list(adata1.obs['Id'].unique())
color_palette = px.colors.qualitative.Set1[:len(cell_types)]
color_dict = {ct: color for ct, color in zip(cell_types, color_palette)}


fig = go.Figure(data=[go.Scatter3d(
    x=adata1.obsm['X_umap_3d'][:, 0],
    y=adata1.obsm['X_umap_3d'][:, 1],
    z=adata1.obsm['X_umap_3d'][:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color=[color_dict.get(cell_type, '#000000') for cell_type in adata1.obs['Id']],
        opacity=0.8
    ),
    text=adata1.obs['Id'],
    hoverinfo='text'
)])
# Mettez à jour la mise en page
fig.update_layout(
    scene=dict(aspectmode="data"),
    title="3D UMAP "
)

fig.show()

# Umap with imputation knn, normalization pearson and scanorama

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
adatas_red = []
data_dir = 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/Formated_data'

for filename in os.listdir(data_dir):
    if filename.endswith('.h5ad'):
            adata = sc.read_h5ad(os.path.join(data_dir, filename))
            adata.var_names_make_unique()
            adatas_red.append(adata)

print(f"Nombre de fichiers h5ad chargés avec succès : {len(adatas_red)}")

In [ ]:
for ade in adatas_red:
    X = ade.X
    knn_imputer = KNNImputer(n_neighbors=5)  
    X_imputed = knn_imputer.fit_transform(X)
    ade.X = X_imputed

In [ ]:
plot_data = []

for ade in adatas_red:
    ad = ade.copy()
    #ad.X = np.nan_to_num(ad.X, nan=0.0)  
    analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(ad, inplace=False)
    
    X_fixed = analytic_pearson["X"]
    
    plot_data.extend([(x, ad.obs['Id'][0]) for x in csr_matrix(X_fixed).sum(1).A1])

df_plot = pd.DataFrame(plot_data, columns=["Normalized Sum", "Dataset"])

plt.figure(figsize=(10, 6))
ax = sns.histplot(data=df_plot, x="Normalized Sum", hue="Dataset", bins=100, kde=False, palette="tab10")

plt.xlabel("Sum of the normalized values")
plt.ylabel("Number of cells")
plt.title("Distribution of the Pearson Residue Normalization")

# Afficher le plot
plt.show()

In [ ]:
adata3 = adt.concat(adatas_red, join="outer", index_unique="-", merge="unique")
adata3.layers['raw'] = adata3.X
adata3.X = np.nan_to_num(adata3.X, nan=0.0)

In [ ]:
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(adata3, inplace=False)
X_fixed = analytic_pearson["X"]
X_fixed[np.isnan(X_fixed)] = 0 
adata3.layers["analytic_pearson_residuals_train"] = csr_matrix(X_fixed)
adata3.X = adata3.layers["analytic_pearson_residuals_train"]
sc.pp.pca(adata3)
sce.pp.scanorama_integrate(adata3, 'Id', verbose =1)

In [ ]:
sc.pl.pca_variance_ratio(adata3, log=True)

In [ ]:
sc.pp.neighbors(adata3, use_rep = 'X_scanorama')
sc.tl.leiden(adata3, resolution = 2, key_added = 'overcluster')
sc.tl.umap(adata3)
sc.pl.umap(
    adata3,
    color=['overcluster']
)

In [ ]:
np.random.seed(1)
ri = np.random.permutation(list(range(adata3.shape[0])))

sc.pl.umap(adata3[ri,:], color = ['Id'], vmin = .5, size = 20)

In [ ]:
#ciliated
sc.pl.umap(adata3, color=['TPPP3_HUMAN','TBB4B_HUMAN','SNTAN_HUMAN','CCD17_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px 

def umap_3D(adata):

    umap_3d = umap.UMAP(n_components=3, random_state=42)

    adata.obsm['X_umap_3d'] = umap_3d.fit_transform(adata.obsm['X_scanorama'])
    
    cell_types = list(adata.obs['Id'].unique())
    color_palette = px.colors.qualitative.Set1[:len(cell_types)]
    color_dict = {ct: color for ct, color in zip(cell_types, color_palette)}


    fig = go.Figure(data=[go.Scatter3d(
        x=adata.obsm['X_umap_3d'][:, 0],
        y=adata.obsm['X_umap_3d'][:, 1],
        z=adata.obsm['X_umap_3d'][:, 2],
        mode='markers',
        marker=dict(
            size=5,
            color=[color_dict.get(cell_type, '#000000') for cell_type in adata.obs['Id']],
            opacity=0.8
        ),
        text=adata.obs['Id'],
        hoverinfo='text'
    )])
    # Mettez à jour la mise en page
    fig.update_layout(
        scene=dict(aspectmode="data"),
        title="3D UMAP "
    )

    fig.show()

In [ ]:
umap_3D(adata3)

In [ ]:
explained_variance = adata3.uns["pca"]["variance_ratio"] * 100  # Conversion en pourcentage
for i, var in enumerate(explained_variance[:10]):  # Afficher les 10 premières composantes
    print(f"PC{i+1}: {var:.2f}% de variance expliquée")

In [ ]:
sc.pl.pca_scatter(adata3, color="Id")

# Umap sans normalisation et avec scanorama

In [ ]:
adatas_red = []
data_dir = 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/Formated_data'

for filename in os.listdir(data_dir):
    if filename.endswith('.h5ad'):
            adata = sc.read_h5ad(os.path.join(data_dir, filename))
            adata.var_names_make_unique()
            adatas_red.append(adata)

print(f"Nombre de fichiers h5ad chargés avec succès : {len(adatas_red)}")

In [ ]:
adata2 = adt.concat(adatas_red, join="outer", index_unique="-", merge="unique")
adata2.layers['raw'] = adata2.X
adata2.X = np.nan_to_num(adata2.X, nan=0.0)

In [ ]:
sc.pp.pca(adata2)
sce.pp.scanorama_integrate(adata2, 'Id', verbose =1)

In [ ]:
sc.pp.neighbors(adata2, use_rep = 'X_scanorama')
sc.tl.leiden(adata2, resolution = 2, key_added = 'overcluster')
sc.tl.umap(adata2)
sc.pl.umap(
    adata2,
    color=['overcluster','Id']
)

In [ ]:
#secretory club
sc.pl.umap(adata2, color=['TRFL_HUMAN','CADH1_HUMAN','MUC5A_HUMAN','Id'],legend_loc="on data",
)

In [ ]:
np.random.seed(1)
ri = np.random.permutation(list(range(adata2.shape[0])))

sc.pl.umap(adata2[ri,:], color = ['Id','overcluster'], vmin = .5, size = 20)

In [ ]:
#ciliated
sc.pl.umap(adata2, color=['TPPP3_HUMAN','TBB4B_HUMAN','SNTAN_HUMAN','CCD17_HUMAN','Id'],legend_loc="on data",
)

## MNN test with Pearson residue 

In [ ]:
import s

In [ ]:
import mnnpy

In [ ]:
%pip install pydeseq2

In [ ]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [ ]:
adata.X

In [ ]:
adata.obs

In [ ]:
adata.X

In [ ]:
counts = pd.DataFrame(adata.X.astype(int), columns = adata.var_names)
dds = DeseqDataSet(
        counts = counts,
        metadata=adata.obs,
        design_factors="Id")
dds.deseq2()
stat_res = DeseqStats(dds, contrast=('Id', 'GFP.NEG3dpi_pg', 'Mock6dpi_pg'))
print(stat_res.summary())

In [ ]:
adata.X = adata.layers['raw_woutnan']

In [ ]:
adata

In [ ]:
adata.X.min()

In [ ]:
mean = np.mean(adata.X, axis=0)
variance = np.var(adata.X, axis=0)

# Afficher la moyenne et la variance pour chaque protéine
print("Moyenne : ", mean)
print("Variance : ", variance)

In [ ]:
plt.hist(adata.X[:, 9], bins=50, density=True, alpha=0.75, color='blue')
plt.title("Histogramme des valeurs pour la protéine 1")
plt.xlabel("Valeurs")
plt.ylabel("Fréquence")
plt.show()

In [ ]:
# Calcul du ratio variance/moyenne
dispersion_ratio = variance / mean

# Afficher les ratios de dispersion
print("Ratios de dispersion : ", dispersion_ratio)

# Vérifier si certains ratios sont > 1, ce qui indique une surdispersion
surdispersion = np.sum(dispersion_ratio > 10)
print(f"Nombre de protéines avec surdispersion : {surdispersion}")

In [ ]:
from scipy.stats import nbinom
X = adata.X
# Exemple pour ajuster la distribution négatif binomiale à une protéine spécifique
# (ici, on prend la première colonne de données)
data = X[:, 3]

# Estimation des paramètres pour la distribution négatif binomiale
mean_data = np.mean(data)
var_data = np.var(data)
size = var_data / (mean_data - var_data)  # Paramètre de dispersion
p = mean_data / var_data  # Paramètre de probabilité

# Tracer la distribution théorique négatif binomiale
x = np.arange(0, np.max(data))
pmf = nbinom.pmf(x, size, p)
plt.hist(data, bins=50, density=True, alpha=0.75, color='blue')
plt.plot(x, pmf, 'r-', label='Distribution négatif binomiale')
plt.title("Ajustement à la distribution négatif binomiale")
plt.xlabel("Valeurs")
plt.ylabel("Fréquence")
plt.legend()
plt.show()

In [ ]:
# Transformation logarithmique des données pour les rendre plus comparables
data_log = np.log1p(data)  # log(1+x) pour éviter des problèmes avec les valeurs 0

# Calculer à nouveau la moyenne et la variance des données transformées
mean_log = np.mean(data_log)
var_log = np.var(data_log)

# Ajuster à la distribution négatif binomiale avec les données transformées
size_log = var_log / (mean_log - var_log)  # Paramètre de dispersion
p_log = mean_log / var_log  # Paramètre de probabilité

# Tracer l'histogramme des données transformées
x_log = np.arange(0, np.max(data_log))
pmf_log = nbinom.pmf(x_log, size_log, p_log)
plt.hist(data_log, bins=50, density=True, alpha=0.75, color='blue')
plt.plot(x_log, pmf_log, 'r-', label='Distribution négatif binomiale ajustée')
plt.title("Ajustement à la distribution négatif binomiale (données transformées)")
plt.xlabel("Valeurs transformées (log)")
plt.ylabel("Fréquence")
plt.legend()
plt.show()

In [ ]:
x_limited = np.arange(0, 50)  # Ajustez cette plage selon vos données
pmf_limited = nbinom.pmf(x_limited, size, p)

# Tracer les données
plt.hist(data, bins=50, density=True, alpha=0.75, color='blue')
plt.plot(x_limited, pmf_limited, 'r-', label='Distribution négatif binomiale ajustée')
plt.title("Ajustement à la distribution négatif binomiale (données limitées)")
plt.xlabel("Valeurs")
plt.ylabel("Fréquence")
plt.legend()
plt.show()